In [ ]:
# %%
'''
00:00 Intro
00:15 Criminal activities public data sets
02:17 Loading data from csv and initial discovery
08:00 Identifying different types of offences
16:54 Sampling on specific types of offences
39:03 Focusing on assult of public servant
44:19 Distribution of offences by district/precinct
52:35 Independence test between offence type and district
'''

In [ ]:

# %%  02:17  Loading data from excel and initial discovery

import pathlib
import warnings
import pandas as pd
warnings.filterwarnings('ignore')

file_name = "\\data\\criminal_files_detailed.csv"
relative_path = str(pathlib.Path().resolve())
crime = pd.read_csv(relative_path + file_name, encoding='cp1255')  # load into DataFrame object

crime.tail(n=2)


In [ ]:
# %%  08:00  Identifying different types of offences

import numpy as np

# see unique num of crime types
crime['StatisticCrimeType'].nunique()

# before we can sum on TikimSum we need to clean up its mess. its actually a columns
# of 'objects' which are in this case strings - we want all of them to be neat integers
# for our case. problem is - some strings have a comma in them which prevents successful
# casting
crime['TikimSum'] = crime['TikimSum'].apply(lambda x: x.strip().replace(',', '') if isinstance(x, str) else x)
crime['TikimSum'] = crime['TikimSum'].astype(int)

# see top 50 crimes. we need to sum accross quarter and location
crime_sum = crime.groupby('StatisticCrimeType')['TikimSum'].sum().rename('total_offences').sort_values(ascending=False)
crime_sum[:50]  # this is a series


In [ ]:
# %%  16:54  Focusing on specific types of offences

# a bunch of 'str.contains' matches separated by or - like the strdetect in the blog
filter_words = ['התפרצות', 'קטטות', 'מרמה', 'עובד ציבור']
selected_offences = crime[crime['StatisticCrimeType'].str.contains('|'.join(filter_words), na=False)]
selected_offences.head(10)

In [ ]:
# a bunch of exact matches separated by or - as in blog
filter_words = ['התפרצות לבתי עסק ומוסדות',
                'התפרצות לבית דירה',
                'קטטות והפרעות ברחובות',
                'מרמה ועושק']
selected_offences = crime[crime['StatisticCrimeType'].isin(filter_words)]
selected_offences.head(10)


In [ ]:
# sum of offences per quarter and type
offences_over_time = selected_offences.groupby(['Quarter', 'StatisticCrimeType'])['TikimSum'].sum().rename('num_offences')

In [ ]:
# in the blog we plotted offences per quarter, stacked bars
offences_over_time.reset_index()[['Quarter', 'StatisticCrimeType', 'num_offences']] \
  .pivot_table(index='Quarter', columns='StatisticCrimeType') \
  .plot(kind='bar', stacked=True, figsize=(12, 8))


In [ ]:
import matplotlib.pyplot as plt

# the legend on the previous plot had bad labels, we can only edit it by using
# using lower methods at matplotlib
_, ax = plt.subplots()
offences_over_time_plot = offences_over_time.reset_index()[['Quarter', 'StatisticCrimeType', 'num_offences']] \
  .pivot_table(index='Quarter', columns='StatisticCrimeType')
offences_over_time_plot.plot(kind='bar', stacked=True, figsize=(12, 8), ax=ax)

labels = [name[::-1] for name in offences_over_time_plot.columns.get_level_values('StatisticCrimeType')]
ax.legend(labels)


In [ ]:
import matplotlib.pyplot as plt

# the legend on the previous plot had bad labels, we can only edit it by using
# using lower methods at matplotlib
fig, ax = plt.subplots(2, 2, figsize=(16, 12))
offences_over_time_plot = offences_over_time.reset_index()[['Quarter', 'StatisticCrimeType', 'num_offences']] \
  .pivot_table(index='Quarter', columns='StatisticCrimeType')

# fix labels (drop the 'num_columns' from the label, reverse hebrew offence names)
offences_over_time_plot = offences_over_time_plot.droplevel(level=0, axis=1)
offences_over_time_plot.rename(columns={name:name[::-1] for name in offences_over_time_plot.columns}, inplace=True)

offences_over_time_plot.plot(kind='bar', subplots=True, figsize=(12, 8), ax=ax, color='black')
fig.tight_layout()
plt.show()


In [ ]:
# %%  39:03  Focusing on assult of public servant

selected_offences = crime[crime['StatisticCrimeType'] == 'תקיפת עובדי צבור בתפקיד']
offences_over_time = selected_offences.groupby(['Quarter', 'StatisticCrimeType'])['TikimSum'].sum().rename('num_offences')

offences_over_time.reset_index()[['Quarter', 'StatisticCrimeType', 'num_offences']] \
  .pivot_table(index='Quarter', columns='StatisticCrimeType') \
  .plot(kind='bar', legend=False, title='תקיפת עובדי צבור בתפקיד'[::-1])


In [ ]:
# %%  44:19  Distribution of offences by district/precinct

# we take all the previous 5 offences, like in the blog
filter_words = ['התפרצות לבתי עסק ומוסדות',
                'התפרצות לבית דירה',
                'קטטות והפרעות ברחובות',
                'מרמה ועושק',
                'תקיפת עובדי צבור בתפקיד']
selected_offences = crime[crime['StatisticCrimeType'].isin(filter_words)]
num_offences_grp = selected_offences.groupby(['PoliceDistrict', 'StatisticCrimeType'])['TikimSum'].sum().rename('num_offences')
num_offences = num_offences_grp.reset_index()[['PoliceDistrict', 'StatisticCrimeType', 'num_offences']]
num_offences_piv = num_offences.pivot_table(index='PoliceDistrict', columns='StatisticCrimeType')

# the columns are a pair of labels - <offence, 'num_offences'> --> drop the level
# 'num_offences' because its same everywhere and messes the legend in the plot
num_offences_piv.columns = num_offences_piv.columns.droplevel(0)

# cheat and reverse index labels for right-to-left plotting
num_offences_piv.columns = [label[::-1] for label in num_offences_piv.columns]
num_offences_piv.index = [label[::-1] for label in num_offences_piv.index]

num_offences_piv.plot(kind='bar', stacked=True, figsize=(12, 8))

In [ ]:
# bars plot - normalized
# ----------------------

# there is no nice way to normalize like in R (seen in the blog video) for this
# kind of plot with pandas, so we'll calc percentage and plot that
num_offences_grp_pct = \
    (num_offences.groupby(['PoliceDistrict', 'StatisticCrimeType'])['num_offences'].sum().rename('num_offences')/num_offences.groupby(['PoliceDistrict'])['num_offences'].sum().rename('num_offences'))
num_offences_pct = num_offences_grp_pct.reset_index()

# the reset is the same as for non-normalized
num_offences_pct_piv = num_offences_pct.pivot_table(index='PoliceDistrict', columns='StatisticCrimeType')
num_offences_pct_piv.columns = num_offences_pct_piv.columns.droplevel(0)                # remove redundant column label
num_offences_pct_piv.columns = [label[::-1] for label in num_offences_pct_piv.columns]  # handle hebrew
num_offences_pct_piv.index = [label[::-1] for label in num_offences_pct_piv.index]      # handle hebrew
num_offences_pct_piv.plot(kind='bar', stacked=True, figsize=(12, 8))

In [ ]:
# %%  52:35  Independence test

#tot_offences_overall = selected_offences.

num_offences_grp

# lets perform a chi-square test to see if there is coupling between type of offence and district
#selected_offences.head()
#crosstab = pd.crosstab(selected_offences["PoliceDistrict"], selected_offences["StatisticCrimeType"])
#crosstab.head()

#stats.chi2_contingency(crosstab)